In [ ]:
!pip list | grep crew

In [ ]:
import os
import sys
import datetime
import pprint

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

In [ ]:
from models.llm import LLM

model = LLM('gpt-4o-mini')
llm = model.load_model()
llm

In [4]:
os.environ["OTEL_SDK_DISABLED"] = "true"
os.environ["CREWAI_TELEMETRY_OPT_OUT"] = "true"

### Search Tools

#### Tavily Search

1000 search credit / month

In [5]:
from langchain_community.tools import TavilySearchResults

tavily_search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=False,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)


In [ ]:
results = tavily_search.invoke({"query": "What happened with the SENSEX yesterday?"})
pprint.pp(results)

#### Serper Search

Total: 2500 search credit free

In [7]:
from langchain_community.utilities import GoogleSerperAPIWrapper

# serper_search = GoogleSerperAPIWrapper()

In [8]:
# results = serper_search.results("Studies on cardiovascular impacts of weight loss drug from Lilly?")


### Crew with single agent and single task

In [ ]:
from crewai import Agent, Task, Crew

# Instantiate tools
search_tool = tavily_search

# Create agents
research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  tools=[search_tool],
  verbose=True
)

# Define tasks
task = Task(
  description='Find and summarize the latest AI news',
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent,
)

# Assemble a crew
crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True,
    iterations=3
)

# Execute tasks
result = crew.kickoff()
print(result)

#### Crew With Planning

In [ ]:
import os
from crewai import Agent, Task, Crew

# Instantiate tools
search_tool = tavily_search

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI in Healthcare industry and AI in Finance industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool],
    llm=llm,
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI in Healthcare industry and AI in Finance industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments with a unique perspective on their significance.',
    agent=researcher
)


# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher],
    tasks=[research],
    llm=llm,
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
result = crew.kickoff()

print(result)

In [ ]:
print(result)